# Create raster dataset from table

In [1]:
import os
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import Affine

In [60]:
# apply multiplicative factors at transmissivity
def raster_from_table(data, table, n=18):
	"""
	data: raster map
	table: table of values in each column (dataframe)
	"""
	shape = data.shape
	grid_size = len(data.reshape(-1))
	data = data.reshape(-1)
	raster = np.full(grid_size, -9999, dtype=float)
	for ilitho in range(np.min([n, len(table.columns)])):
		# identify cells with lithological units
		idlitho = np.where(data == ilitho)[0]
		# get values of transmissivity
		ifactor = table['litho_'+str(ilitho)][0]
		# assign transmisivity
		raster[idlitho] = ifactor
	return raster.reshape(shape)
	
def raster_from_factors(data, raster, factors, n=19):
	"""
	data: raster map
	factors: table of values in each column (dataframe)
	"""
	shape = data.shape
	grid_size = len(data.reshape(-1))
	data = data.reshape(-1)
	raster = raster.reshape(-1)
	for ilitho in range(np.min([n, len(factors.columns)])):
		# identify cells with lithological units
		idlitho = np.where(data == ilitho)[0]
		# get values of transmissivity
		ifactor = factors['litho_'+str(ilitho)][0]
		# assign transmisivity
		raster[idlitho] = raster[idlitho]*ifactor		
	return raster.reshape(shape)

def save_raster(fname, data, profile, transform):
	os.remove(fname) if os.path.exists(fname) else None
	with rasterio.open(fname, 'w', **profile) as dst:
		# Write the modified raster data
		dst.write(np.array(data, dtype=np.float32), 1)
		# Set the affine transformation
		dst.transform = transform

def open_raster(fname):
	with rasterio.open(fname) as src:
		# Read the input raster data
		data = src.read(1)
		# Get the metadata of the input raster
		profile = src.profile
		# Get the affine transformation
		transform = src.transform
	return data, profile, transform

def get_mask(fmask):
	# output an array
	# get a mask
	mask = rasterio.open(fmask).read(1)
	# mask values for visualisation
	mask = np.array(mask, dtype=float)
	mask[mask <= 0] = np.nan
	return mask


## Create a raster dataset from table

##### Read factor/transmissivity dataset or create one

In [47]:
#dftable = pd.read_csv(fcsv_par)

In [65]:
dftable = pd.DataFrame({"litho_0":[0.1],
                  "litho_1":[0.5],
                  "litho_2":[0.5]})

In [66]:
dftable

,litho_0,litho_1,litho_2
0,0.1,0.5,0.5


##### Open a raster file

In [67]:
ftypes = "C:/Users/Edisson/OneDrive - Cardiff University/models/cuwalid/examples_json/multiaq/input/MA_aqtype1.asc"

In [68]:
# open raster
litho, profile, transform = open_raster(ftypes)

##### Create new raster dataset

In [69]:
# create raster
data = raster_from_table(litho, dftable)

#### Save new raster dataset

In [70]:
# save raster
fname_out = "C:/Users/Edisson/OneDrive - Cardiff University/models/cuwalid/examples_json/multiaq/input/MA_test_ksat.asc"
save_raster(fname_out, data, profile, transform)

## Modify a raster dataset using factors from a table

##### Read factor/transmissivity dataset or create one

In [54]:
dftable = pd.DataFrame({"litho_0":[0.5],
                  "litho_1":[0.5],
                  "litho_2":[0.5]})

In [55]:
dftable

,litho_0,litho_1,litho_2
0,0.5,0.5,0.5


##### Open a raster file

Read raster dataset containing ids for regions/zones

In [56]:
ftypes = "C:/Users/Edisson/OneDrive - Cardiff University/models/cuwalid/examples_json/multiaq/input/MA_aqtype1.asc"

Read raster dataset to modify

In [57]:
fraster = "C:/Users/Edisson/OneDrive - Cardiff University/models/cuwalid/examples_json/multiaq/input/MA_test_ksat.asc"

In [58]:
# open raster
litho, profile, transform = open_raster(ftypes)
raster_in = open_raster(fraster)[0]

##### Create new raster dataset

In [61]:
# create raster
data = raster_from_factors(litho, raster_in, dftable)

#### Save new raster dataset

In [62]:
# save raster
fname_out = "C:/Users/Edisson/OneDrive - Cardiff University/models/cuwalid/examples_json/multiaq/input/MA_test_ksat_mod.asc"
save_raster(fname_out, data, profile, transform)